In [ ]:

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import base64

import pandas as pd
import numpy as np
import re
#from sklearn.externals import joblib

import joblib as jb
import matplotlib.pyplot as plt

import ast
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD

from scipy import sparse
from lightfm import LightFM
from lightfm.evaluation import auc_score
from lightfm.evaluation import precision_at_k, recall_at_k
import pickle

#image_filename = 'skimage.png'
#encoded_image = base64.b64encode(open(image_filename, 'rb').read())

skin = pd.read_csv('../data/skindata.csv', index_col=[0])

#with open('mf_model.pkl', 'rb') as f:
#       mf_model = pickle.load(f)

def dicts(skin, colname):
    vals = list(set(skin[colname]))
    l = []
    for i in vals:
        dic = {}
        dic['label'] = i
        dic['value'] = i
        l.append(dic)
    return l

tones_dict = dicts(skin, 'Skin_Tone')
types_dict = dicts(skin, 'Skin_Type')
eyes_dict = dicts(skin, 'Eye_Color')
hair_dict = dicts(skin, 'Hair_Color')

products_dictionary = dicts(skin, 'Product')

user_dictionary = dicts(skin, 'User_id')

f=open('mf_model.pkl','wb')
pickle.dump(dicts,f)
f.close() #Adding a close after closing will not give an error

with open('mf_model.pkl', 'rb') as f:
    mf_model = pickle.load(f)

def Table(skin):
    rows = []
    for i in range(len(skin)):
        row = []
        for col in skin.columns:
            value = skin.iloc[i][col]
            # update this depending on which
            # columns you want to show links for
            # and what you want those links to be
            if col == 'Product':
                cell = html.Td(html.A(href=skin.iloc[i]['Product_Url'], children=value))
            elif col == 'Product_Url':
                continue
            else:
                cell = html.Td(children=value)
            row.append(cell)
        rows.append(html.Tr(row))
    return html.Table(
        # Header
        [html.Tr([html.Th(col) for col in ['Rating', 'Product']])] + rows
        )

separation_string = '''
'''

intro_text = '''
__Welcome to skincare recommendations! Whether you are new to the world of skincare, or already have your favorite products__,
__one of the recommenders below can help. Our first recommender uses your personal features to estimate what products may work best for you__,
__and the second one requires you to select a product that you already like, and the system will give you the names of similar items. Both of them__
_
__
'''


markdown_text_1 = '''
__Based on your features, these are the top products for you:__
'''

markdown_text_2 = '''
__Based on your preference, these are the top products for you:__
'''

markdown_text_3 = '''
__This user may like the following products:__
'''

def create_interaction_matrix(skin, user_col, item_col, rating_col, norm= False, threshold = None):
    interactions = skin.groupby([user_col, item_col])[rating_col].sum().unstack().reset_index().fillna(0).set_index(user_col)
    if norm:
        interactions = interactions.applymap(lambda x: 1 if x > threshold else 0)
    return interactions

interaction_matrix = create_interaction_matrix(skin=skin, user_col='User_id', item_col = 'Product_id', rating_col='Rating_Stars')

def create_user_dict(interactions):
    user_id = list(interactions.index)
    user_dict = {}
    counter = 0
    for i in user_id:
        user_dict[i] = counter
        counter += 1
    return user_dict

user_dict = create_user_dict(interaction_matrix)

def create_item_dict(skin, id_col, name_col):
    item_dict ={}
    for i in skin.index:
        item_dict[(skin.loc[i, id_col])] = skin.loc[i, name_col]
    return item_dict

product_dict = create_item_dict(skin = skin, id_col = 'Product_id', name_col = 'Product')

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

#https://codepen.io/chriddyp/pen/bWLwgP.css

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)



colors = {
    #'background': '#1DB954',
    "text": "#111111",
    "background-image" : "url('/assets/wallpaperskin_retouched.png')",
    "background-size": "cover",
}

app.layout = html.Div(style=colors, children=[
    html.H1(children='Skincare Recommendations',
			style={
				'textAlign': 'center',
				'color': colors['text'],
				'backgroundColor': colors["background-image"],
                 'font-family': 'Bangers'
			}
	),


        dcc.Markdown(children=intro_text),

        dcc.Markdown(children=separation_string),


	html.Label('Skin Tone'),
    dcc.Dropdown(
		id='skintone-selector',
        options=tones_dict,
		placeholder='Select your skin tone'
    ),

	html.Label('Skin Type'),
    dcc.Dropdown(
		id='skintype-selector',
        options=types_dict,
        placeholder='Select your skin type'
    ),

	html.Label('Eye color'),
    dcc.Dropdown(
		id='eyecolor-selector',
		options=eyes_dict,
        placeholder='Select your eye color'
    ),

	html.Label('Hair color'),
    dcc.Dropdown(
		id='haircolor-selector',
        options=hair_dict,
        placeholder='Select your eye color'
    ),

    dcc.Markdown(children=separation_string),

	dcc.Markdown(children=markdown_text_1),

    html.Div(id='output_1'),

    dcc.Markdown(children=separation_string),

    html.H2(children='Skincare recommendations based on your favorites',
			style={
				'textAlign': 'center',
				'color': colors['text'],
				'backgroundColor': colors["background-image"],
                'font-family': 'Bangers'
			}
	),
    html.Label('Your favorite product!'),
    dcc.Dropdown(
		id='product-selector',
        options=products_dictionary,
		placeholder='Select your favorite product'
    ),

    dcc.Markdown(children=markdown_text_2),

    html.Div(id='output_2'),

    dcc.Markdown(children=separation_string),

    ])

@app.callback(
	Output('output_1', 'children'),
    [Input('skintone-selector', 'value'),
    Input('skintype-selector', 'value'),
    Input('eyecolor-selector', 'value'),
    Input('haircolor-selector', 'value')])

def recommend_products_by_user_features(skintone, skintype, eyecolor, haircolor):
    ddf = skin[(skin['Skin_Tone'] == skintone) & (skin['Hair_Color'] == haircolor) & (skin['Skin_Type'] == skintype) & (skin['Eye_Color'] == eyecolor)]
    recommendations = ddf[(skin['Rating_Stars'].notnull())]
    data = recommendations[['Rating_Stars', 'Product_Url', 'Product','Category']]

    data = data.sort_values('Rating_Stars', ascending=False).head()

    return Table(data)

@app.callback(
	Output('output_2', 'children'),
    [Input('product-selector', 'value')]
    )

def content_recommender(product):
    try:
        skin_ing = skin[['Product', 'Product_id', 'Ingredients', 'Product_Url', 'Ing_Tfidf', 'Rating','Category']]
        skin_ing.drop_duplicates(inplace=True)
        skin_ing = skin_ing.reset_index(drop=True)
        tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
        tfidf_matrix = tf.fit_transform(skin_ing['Ingredients'])
        cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
        titles = skin_ing[['Product', 'Ingredients', 'Rating', 'Product_Url','Category']]
        indices = pd.Series(skin_ing.index, index=skin_ing['Product'])
        idx = indices[product]
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:11]
        product_indices = [i[0] for i in sim_scores]

    except KeyError:
        return None

    return Table(titles.iloc[product_indices])

if __name__ == '__main__':
    app.run_server(debug=False)


c:\users\gurur\appdata\local\programs\python\python37\lib\site-packages\lightfm\_lightfm_fast.py:10: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  "LightFM was compiled without OpenMP support. "


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Dec/2020 12:35:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Dec/2020 12:35:37] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [19/Dec/2020 12:35:37] "GET /_dash-layout HTTP/1.1" 200 -
c:\users\gurur\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:237: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
127.0.0.1 - - [19/Dec/2020 12:35:37] "POST /_dash-update-component HTTP/1.1" 200 -
c:\users\gurur\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:252: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
127.0.0.1 - - [19/Dec/2020 12:35:37] "GET /assets/wallpaperskin_retouched.png HTTP/1.1" 404 -
127.0.0.1 - - [19/Dec/2020 12:35:37] "POST /_das

In [ ]:
%tb